In [2]:
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt

import numpy as np
import math

import cv2

from keras import layers
import keras

#See: https://github.com/ethanyanjiali/deep-vision/blob/master/Hourglass/tensorflow/hourglass104.py

c:\Users\playf\OneDrive\Documents\UBC\Thesis\Custom_Model_Code\custom_model_venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#Data Pre-processing Steps
#Creating lists of the images
base_dir='C:/Users/playf/OneDrive/Documents/UBC/Thesis/Custom_Model_Code/data/labels/'
train_paths=np.loadtxt(base_dir+'keypoints_spaces_train.txt',usecols=0,dtype=str)
valid_paths=np.loadtxt(base_dir+'keypoints_spaces_valid.txt',usecols=0,dtype=str)
test_paths=np.loadtxt(base_dir+'keypoints_spaces_test.txt',usecols=0,dtype=str)

train_images=[]
valid_images=[]
test_images=[]
for path in train_paths:
    img=cv2.imread(path)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img_norm=cv2.normalize(img,None,0,1.0,cv2.NORM_MINMAX,dtype=cv2.CV_32F)
    train_images.append(img_norm)

for path in valid_paths:
    img=cv2.imread(path)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img_norm=cv2.normalize(img,None,0,1.0,cv2.NORM_MINMAX,dtype=cv2.CV_32F)
    valid_images.append(img_norm)

for path in test_paths:
    img=cv2.imread(path)
    img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    img_norm=cv2.normalize(img,None,0,1.0,cv2.NORM_MINMAX,dtype=cv2.CV_32F)
    test_images.append(img_norm)


#Converting image lists into a (num_images,512,512,1) np.array
train_images=np.array(train_images,dtype=np.float32)
num_train=np.shape(train_images)[0]
train_images=np.expand_dims(train_images,axis=-1)
train_images=train_images.reshape(num_train,512,512,1)
print(np.shape(train_images))

test_images=np.array(test_images,dtype=np.float32)
num_test=np.shape(test_images)[0]
test_images=np.expand_dims(test_images,axis=-1)
test_images=test_images.reshape(num_test,512,512,1)
print(np.shape(test_images))

valid_images=np.array(valid_images,dtype=np.float32)
num_valid=np.shape(valid_images)[0]
valid_images=np.expand_dims(valid_images,axis=-1)
valid_images=valid_images.reshape(num_valid,512,512,1)
print(np.shape(valid_images))

(647, 512, 512, 1)
(82, 512, 512, 1)
(82, 512, 512, 1)


In [4]:
#Creating np array of the labels
train_keypoints=np.loadtxt(base_dir+'keypoints_spaces_train.txt',usecols=[1,2,3,4,5,6,7,8])
valid_keypoints=np.loadtxt(base_dir+'keypoints_spaces_valid.txt',usecols=[1,2,3,4,5,6,7,8])
test_keypoints=np.loadtxt(base_dir+'keypoints_spaces_test.txt',usecols=[1,2,3,4,5,6,7,8])

In [5]:
#Training data augmentation for better performance (and robustness)
batch_size=32
trainDS = tf.data.Dataset.from_tensor_slices((train_images, train_keypoints))
ds2=trainDS.take(150) #Taking 200 images that we will augment
#Augmentation function
data_aug=keras.Sequential([
    layers.RandomRotation(0.05)
])

ds2=(
    ds2
    .shuffle(batch_size*100)
    .batch(batch_size)
    .map(lambda x, y: (data_aug(x), y),
		 num_parallel_calls=tf.data.AUTOTUNE)
    .prefetch(tf.data.AUTOTUNE)
    
)

train_dataset_full=tf.data.Dataset.sample_from_datasets([trainDS,ds2])

In [6]:
def build_keypoint_detection_model(input_shape, num_keypoints=8):
    # Create a custom input layer for grayscale images
    inputs = tf.keras.Input(shape=input_shape)

    # Convert single-channel input to three channels (RGB)
    x = layers.Concatenate()([inputs, inputs, inputs])

    base_model = tf.keras.applications.MobileNetV2(input_tensor=x, alpha=1.0, include_top=False, weights='imagenet')
    x = base_model.output

    # Add custom head for regression
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.5)(x)

    # Output layer for keypoints (8 floating-point values)
    outputs = layers.Dense(num_keypoints)(x)

    model = tf.keras.Model(inputs=inputs, outputs=outputs, name='keypoint_detection_model')
    return model

In [7]:
#Defining variables
input_shape=(512,512,1) #Input image is a 512x512 grayscale image
num_keypoints=8
#Training settings
epochs=240
batch_size=32

In [8]:
#Data augmentation
data_aug=keras.Sequential([
    layers.RandomRotation(0.05)
])
trainDS = tf.data.Dataset.from_tensor_slices((train_images, train_keypoints))
trainDS=(
    trainDS
    .shuffle(batch_size*100)
    .batch(batch_size)
    .map(lambda x, y: (data_aug(x), y),
		 num_parallel_calls=tf.data.AUTOTUNE)
    .prefetch(tf.data.AUTOTUNE)
    
)

In [9]:
model =build_keypoint_detection_model(input_shape,num_keypoints=num_keypoints)
model.summary()
model.compile(optimizer='adam', loss='mean_squared_logarithmic_error', metrics=['mae','msle','mse'])


Model: "keypoint_detection_model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 512, 512, 1)]        0         []                            
                                                                                                  
 concatenate (Concatenate)   (None, 512, 512, 3)          0         ['input_1[0][0]',             
                                                                     'input_1[0][0]',             
                                                                     'input_1[0][0]']             
                                                                                                  
 Conv1 (Conv2D)              (None, 256, 256, 32)         864       ['concatenate[0][0]']         
                                                                           

In [10]:
#Create new epoch loader
#Creating epoch logger
checkpoint_path = "E:/Alexandre_EyeGazeProject/Custom_Model_Code/models/Model_V2/cp-{epoch:04d}.ckpt"
n_batches = np.shape(train_images)[0] / batch_size
n_batches = math.ceil(n_batches)
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=25*n_batches) #Save every 25 epochs
#Save the initial epoch
model.save_weights(checkpoint_path.format(epoch=0))
history=model.fit(train_images,
                  train_keypoints,
                  validation_data=(valid_images,valid_keypoints),
                  batch_size=batch_size,
                  epochs=epochs,
                  callbacks=[cp_callback])

#Save the entire (most recent epoch) model:
model.save('E:/Alexandre_EyeGazeProject/Custom_Model_Code/models/Model_V2/eyecorner_Aug2.keras')
model.save('E:/Alexandre_EyeGazeProject/Custom_Model_Code/models/Model_V2/models/eyecorner_Aug2.h5')
#Save the entire (most recent epoch) model:
save_path='E:/Alexandre_EyeGazeProject/Custom_Model_Code/models/Model_V2/FullModel/'
tf.saved_model.save(model,save_path)


Epoch 1/240
21/21 [==============================] - 388s 17s/step - loss: 0.0593 - mae: 0.4680 - msle: 0.0593 - mse: 0.3991 - val_loss: 0.3453 - val_mae: 1.9244 - val_msle: 0.3453 - val_mse: 4.6241
Epoch 2/240
21/21 [==============================] - 305s 15s/step - loss: 0.0238 - mae: 0.1859 - msle: 0.0238 - mse: 0.0570 - val_loss: 1.2740 - val_mae: 3.6645 - val_msle: 1.2740 - val_mse: 20.2703
Epoch 3/240
21/21 [==============================] - 302s 14s/step - loss: 0.0156 - mae: 0.1441 - msle: 0.0156 - mse: 0.0332 - val_loss: 1.6819 - val_mae: 4.1819 - val_msle: 1.6819 - val_mse: 28.5074
Epoch 4/240
21/21 [==============================] - 303s 14s/step - loss: 0.0111 - mae: 0.1188 - msle: 0.0111 - mse: 0.0230 - val_loss: 1.8366 - val_mae: 4.4670 - val_msle: 1.8366 - val_mse: 30.8445
Epoch 5/240
21/21 [==============================] - 302s 14s/step - loss: 0.0092 - mae: 0.1074 - msle: 0.0092 - mse: 0.0198 - val_loss: 1.8883 - val_mae: 4.5690 - val_msle: 1.8883 - val_mse: 30.9482
E

c:\Users\playf\OneDrive\Documents\UBC\Thesis\Custom_Model_Code\custom_model_venv\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


INFO:tensorflow:Assets written to: E:/Alexandre_EyeGazeProject/Custom_Model_Code/models/Model_V2/FullModel/assets


INFO:tensorflow:Assets written to: E:/Alexandre_EyeGazeProject/Custom_Model_Code/models/Model_V2/FullModel/assets
